<a href="https://colab.research.google.com/github/AliBank/InceptionV3---Deep-Learning/blob/main/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q keras
!pip install h5py pyyaml

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.layers import Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from keras.optimizers import RMSprop
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from tensorflow.keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from glob import glob
from keras.utils import to_categorical
import cv2
import numpy as np
from __future__ import absolute_import, division, print_function
import os
import tensorflow as tf
from tensorflow import keras

In [ ]:
train_path="/content/drive/MyDrive/BaklagilDataSet/DB3_Histeq/fold1/train/images/"
test_path="/content/drive/MyDrive/BaklagilDataSet/DB3_Histeq/fold1/test/images/"

In [ ]:
numberOfClass=len(glob(train_path+"/*"))

In [ ]:
print(numberOfClass)

30


In [ ]:
inc= InceptionV3(include_top=False,
                 weights='imagenet',
                 input_shape=(224,224,3))

inc.trainable=False
x =Flatten()(inc.output)
x =GlobalAveragePooling2D()(inc.output)
x= Dense(32, activation='relu')(x)
x= Dense(32, activation='relu')(x)
out=Dense(30,activation='softmax')(x)
model= Model(inputs=inc.inputs, outputs=out, name='Xception')

print(len(model.layers))
for layer in model.layers:
  print(layer.name," ",layer.trainable)

for layer in model.layers[:10]:
  layer.trainable=False
model.summary()  



In [ ]:
#print(inc.summary())

In [ ]:
print(type(inc))

<class 'keras.engine.functional.Functional'>


In [ ]:
#inc_layer_list= inc.layers
#print(inc_layer_list)

In [ ]:
#last_layer = inc.get_layer('mixed10')
#print(last_layer.output_shape)


In [ ]:
#last_output= last_layer.output
#x = layers.Flatten()(last_output)
#x = layers.Dense(1024, activation='relu')(x)
#x = layers.Dropout(0.2)(x)
#x= layers.Dense(1, activation='sigmoid')(x)

#model = Model(inc.input, x)

In [ ]:
print(model)

In [ ]:
#print(model.summary())

In [ ]:
from keras.optimizers.optimizer_v2.rmsprop import RMSProp
optim_1 = RMSProp(lr=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=optim_1, metrics=["accuracy"])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
#train
train_data= ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224))
test_data= ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224))

Found 3601 images belonging to 30 classes.
Found 900 images belonging to 30 classes.


In [ ]:
batch_size=50

In [ ]:
from keras.callbacks import ModelCheckpoint,CSVLogger
filepath="/content/drive/MyDrive/saved_models/weights-improvementIncV3-{epoch:05d}.ckpt"
checkpoint= ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False,save_freq=5*72, mode='max')
#log_csv =CSVLogger('my_logs.csv', separator=',', append=False)
callbacks_list =[checkpoint]

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=100,
                         validation_data= test_data,
                         validation_steps = 900//batch_size )

<ipython-input-14-69e4907365a4>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(train_data,


Epoch 1/100
72/72 [==============================] - 350s 5s/step - loss: 5.4187 - accuracy: 0.0537 - val_loss: 3.6389 - val_accuracy: 0.0556
Epoch 2/100
72/72 [==============================] - 321s 4s/step - loss: 3.4465 - accuracy: 0.0874 - val_loss: 3.3286 - val_accuracy: 0.1094
Epoch 3/100
72/72 [==============================] - 321s 4s/step - loss: 3.2367 - accuracy: 0.1145 - val_loss: 3.1735 - val_accuracy: 0.1128
Epoch 4/100
72/72 [==============================] - 316s 4s/step - loss: 3.1101 - accuracy: 0.1420 - val_loss: 3.0568 - val_accuracy: 0.1597
Epoch 5/100
72/72 [==============================] - 317s 4s/step - loss: 2.9303 - accuracy: 0.1771 - val_loss: 3.0221 - val_accuracy: 0.1701
Epoch 6/100
72/72 [==============================] - 315s 4s/step - loss: 2.8612 - accuracy: 0.1962 - val_loss: 2.9129 - val_accuracy: 0.1667
Epoch 7/100
72/72 [==============================] - 324s 5s/step - loss: 2.7678 - accuracy: 0.2201 - val_loss: 2.8301 - val_accuracy: 0.2049
Epoch 

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=15,
                         initial_epoch=0,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementIncV3-00015.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=25,
                         initial_epoch=15,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementIncV3-00025.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=50,
                         initial_epoch=25,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementIncV3-00050.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=75,
                         initial_epoch=50,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementIncV3-00075.ckpt")

In [ ]:
hist=model.fit_generator(train_data,
                         steps_per_epoch=3600//batch_size,
                         epochs=100,
                         initial_epoch=75,
                         validation_data= test_data,
                         validation_steps = 900//batch_size,
                         callbacks=callbacks_list)

In [ ]:
model.load_weights("/content/drive/MyDrive/saved_models/weights-improvementIncV3-00100.ckpt")

In [ ]:
model.save_weights("BaklagilFold1RMSProplr0.0001InceptionV3")